In [18]:
%matplotlib inline

DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [1]:
from __future__ import division

import sys
import time
import logging

import numpy as np
from scipy.io import loadmat
from os import path

# change this to the path of mushu if you don't have it in your
# PYTHONPATH already
sys.path.append('../../mushu')
sys.path.append('../')

import libmushu
from libmushu.driver.replayamp import ReplayAmp
from wyrm.types import RingBuffer
import wyrm.processing as proc
from wyrm import io
from wyrm.types import Data, BlockBuffer, RingBuffer

#Classifiers
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib
import matplotlib.pyplot as plt

DEBUG:matplotlib:$HOME=C:\Users\aleja
DEBUG:matplotlib:matplotlib data path C:\Users\aleja\Anaconda3\lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:loaded rc file C:\Users\aleja\Anaconda3\lib\site-packages\matplotlib\mpl-data\matplotlibrc
DEBUG:matplotlib:matplotlib version 2.2.2
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'nt', 'winreg', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'encodings.cp1252', 'site', 'os', 'errno', 'stat', '_stat', 'ntpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', 'types', 'functools', '_functools', 'collections', 'operator', '_operator', 'keyword', 'heapq', '_heapq', 'itertools', 'reprl

DEBUG:matplotlib:CACHEDIR=C:\Users\aleja\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\aleja\.matplotlib\fontList.json
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [80]:
logging.basicConfig(format='%(relativeCreated)10.0f %(threadName)-10s %(name)-10s %(levelname)8s %(message)s', level=logging.NOTSET)
logger = logging.getLogger(__name__)

In [2]:
# replay the experiment in real time?
REALTIME = False


TRAIN_DATA = '../BCI_Comp_III_Wads_2004/data/Subject_A_Train.mat'
TEST_DATA = '../BCI_Comp_III_Wads_2004/data/Subject_A_Test.mat'

CHANNEL_DATA = '../BCI_Comp_III_Wads_2004/data/eloc64.txt'

TRUE_LABELS = "WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU"

STIMULUS_CODE = {
    # cols from left to right
    1 : "agmsy5",
    2 : "bhntz6",
    3 : "ciou17",
    4 : "djpv28",
    5 : "ekqw39",
    6 : "flrx4_",
    # rows from top to bottom
    7 : "abcdef",
    8 : "ghijkl",
    9 : "mnopqr",
    10: "stuvwx",
    11: "yz1234",
    12: "56789_"
}

MARKER_DEF_TRAIN = {'target': ['target'], 'nontarget': ['nontarget']}
MARKER_DEF_TEST = {i : [i] for i in STIMULUS_CODE.values()}

SEG_IVAL = [0, 800]

In [83]:
def train(filename):
    cnt = load_bci_data(filename)

    fs_n = cnt.fs / 2

    b, a = proc.signal.butter(5, [10 / fs_n], btype='low')
    cnt = proc.lfilter(cnt, b, a)

#     b, a = proc.signal.butter(5, [.4 / fs_n], btype='high')
#     cnt = proc.lfilter(cnt, b, a)

    cnt = proc.subsample(cnt, 20)

    epo = proc.segment_dat(cnt, MARKER_DEF_TRAIN, SEG_IVAL)

    #from wyrm import plot
    #plot.plot_spatio_temporal_r2_values(proc.sort_channels(epo))
    #print JUMPING_MEANS_IVALS
    #plot.plt.show()

    fv = proc.create_feature_vectors(epo)
    X = fv.data
    y = fv.axes[0]
    clf = proc.lda_train(fv)
#     clf = LinearDiscriminantAnalysis(solver='eigen')
#     clf.fit(X,y)
    return clf

In [4]:
def load_bci_data(filename):
    """Load the BCI Competition III Data Set 2.
    This method loads the data set and converts it into Wyrm's ``Data``
    format. Before you use it, you have to download the data set in
    Matlab format and unpack it. The directory with the extracted files
    must contain the ``Subject_*.mat``- and the ``eloc64.txt`` files.
    .. note::
        If you need the true labels of the test sets, you'll have to
        download them separately from
        http://bbci.de/competition/iii/results/index.html#labels
    Parameters
    ----------
    filename : str
        The path to the matlab file to load
    Returns
    -------
    cnt : continuous `Data` object
    Examples
    --------
    >>> dat = load_bcicomp3_ds2('/home/foo/data/Subject_A_Train.mat')
    """
    STIMULUS_CODE = {
        0 : "blankMatrix",
        # cols from left to right
        1 : "agmsy5",
        2 : "bhntz6",
        3 : "ciou17",
        4 : "djpv28",
        5 : "ekqw39",
        6 : "flrx4_",
        # rows from top to bottom
        7 : "abcdef",
        8 : "ghijkl",
        9 : "mnopqr",
        10: "stuvwx",
        11: "yz1234",
        12: "56789_"
        }

    # load the matlab data
    data_mat = loadmat(filename)
    # load the channel names (the same for all datasets
    eloc_file = path.sep.join([path.dirname(filename), 'eloc64.txt'])
    with open(eloc_file) as fh:
        data = fh.read()
    channels = []
    for line in data.splitlines():
        if line:
            chan = line.split()[-1]
            chan = chan.replace('.', '')
            channels.append(chan)
    # fix the channel names, some letters have the wrong capitalization
    for i, s in enumerate(channels):
        s2 = s.upper()
        s2 = s2.replace('Z', 'z')
        s2 = s2.replace('FP', 'Fp')
        channels[i] = s2
    # The signal is recorded with 64 channels, bandpass filtered
    # 0.1-60Hz and digitized at 240Hz. The format is Character Epoch x
    # Samples x Channels
    data = data_mat['Signal']
    data = data.astype('double')
    # For each sample: 1 if a row/colum was flashed, 0 otherwise
    flashing = data_mat['Flashing'].reshape(-1)
    #flashing = np.flatnonzero((np.diff(a) == 1)) + 1
    ##Creates an array where only the initial intensifications of each series appear
    tmp = []
    for i, _ in enumerate(flashing):
        if i == 0:
            tmp.append(flashing[i])
            continue
        if flashing[i] == flashing[i-1] == 1:
            tmp.append(0)
            continue
        tmp.append(flashing[i])
    flashing = np.array(tmp)
    # For each sample: 0 when no row/colum was intensified,
    # 1..6 for intensified columns, 7..12 for intensified rows
    stimulus_code = data_mat['StimulusCode'].reshape(-1)
    stimulus_code = stimulus_code[flashing == 1]
    # 0 if no row/col was intensified or the intensified did not contain
    # the target character, 1 otherwise
    stimulus_type = data_mat.get('StimulusType', np.array([])).reshape(-1)
    # The target characters
    target_chars = data_mat.get('TargetChar', np.array([])).reshape(-1)
    fs = 240
    data = data.reshape(-1, 64)
    timeaxis = np.linspace(0, data.shape[0] / fs * 1000, data.shape[0], endpoint=False)
    dat = Data(data=data, axes=[timeaxis, channels], names=['time', 'channel'], units=['ms', '#'])
    dat.fs = fs
    # preparing the markers
    target_mask = np.logical_and((flashing == 1), (stimulus_type == 1)) if len(stimulus_type) > 0 else []
    nontarget_mask = np.logical_and((flashing == 1), (stimulus_type == 0)) if len(stimulus_type) > 0 else []
    flashing = (flashing == 1)
    flashing = [[i, 'flashing'] for i in timeaxis[flashing]]
    targets = [[i, 'target'] for i in timeaxis[target_mask]]
    nontargets = [[i, 'nontarget'] for i in timeaxis[nontarget_mask]]
    dat.stimulus_code = stimulus_code[:]
    stim = []
    for i,_ in enumerate(flashing):
        stim.append([flashing[i][0], STIMULUS_CODE[stimulus_code[i]]])
    stimulus_code = stim
    #stimulus_code = zip([t for t, _ in flashing], [STIMULUS_CODE[i] for i in stimulus_code])
    #Raises error "TypeError: '<' not supported between instances of 'tuple' and 'list'" when calling sort() 
    #stimulus_code =[[t for t,_ in flashing], [STIMULUS_CODE[i] for i in stimulus_code]]
    #print(type(stimulus_code), type(flashing), type(targets), type(nontargets))
    markers = flashing[:]
    markers.extend(targets)
    markers.extend(nontargets)
    markers.extend(stimulus_code)
    markers.sort()
    dat.markers = markers[:]
    return dat


In [201]:
def online_experiment(amp, cfy):
    amp_fs = amp.get_sampling_frequency()
    amp_channels = amp.get_channels()

    #buf = BlockBuffer(4)
    rb = RingBuffer(5000)

    fn = amp_fs / 2
    b_low, a_low = proc.signal.butter(5, [10 / fn], btype='low')
#     b_high, a_high = proc.signal.butter(5, [.4 / fn], btype='high')

    zi_low = proc.lfilter_zi(b_low, a_low, len(amp_channels))
#     zi_high = proc.lfilter_zi(b_high, a_high, len(amp_channels))

    amp.start()
    markers_processed = 0
    current_letter_idx = 0
    current_letter = TRUE_LABELS[current_letter_idx].lower()

    letter_prob = {i : 0 for i in 'abcdefghijklmnopqrstuvwxyz123456789_'}
    endresult = []

    t0 = time.time()
    data, markers = amp.get_data()
    while True:
        t0 = time.time()

        # get fresh data from the amp
        data, markers = amp.get_data()
        if len(data) == 0:
            continue

        # we should rather wait for a specific end-of-experiment marker
        if len(data) == 0:
            break

        # convert to cnt
        cnt = io.convert_mushu_data(data, markers, amp_fs, amp_channels)

        ## enter the block buffer
        #buf.append(cnt)
        #cnt = buf.get()
        #if not cnt:
        #    continue

        # low-pass and subsample
        cnt, zi_low = proc.lfilter(cnt, b_low, a_low, zi=zi_low)
#         cnt, zi_high = proc.lfilter(cnt, b_high, a_high, zi=zi_high)

        cnt = proc.subsample(cnt, 20)

        newsamples = cnt.data.shape[0]

        # enter the ringbuffer
        rb.append(cnt)
        cnt = rb.get()

        # segment
        epo = proc.segment_dat(cnt, MARKER_DEF_TEST, SEG_IVAL, newsamples=newsamples)
        if not epo:
            continue

        fv = proc.create_feature_vectors(epo)
        logger.debug('Markers processed: ')
        logger.debug(markers_processed)
#         logger.debug(markers_processed)

        lda_out = proc.lda_apply(fv, cfy)
        markers = [fv.class_names[cls_idx] for cls_idx in fv.axes[0]]
        result = zip(markers, lda_out)
        for s, score in result: #letters in one row/column
#             print(s, score)
            if markers_processed == 180: # 12 trials x 15 repetitions = 180
                endresult.append(sorted(letter_prob.items(), key=lambda x: x[1])[-1][0])
                letter_prob = {i : 0 for i in 'abcdefghijklmnopqrstuvwxyz123456789_'}
                markers_processed = 0
                current_letter_idx += 1
                current_letter = TRUE_LABELS[current_letter_idx].lower()
            for letter in s:
#                 print('letter', letter, '\tscore: ', score)
                letter_prob[letter] += score
            markers_processed += 1
#         print(letter_prob)
#         logger.debug("".join([i[0] for i in sorted(letter_prob.items(), key=lambda x: x[1], reverse=True)]).replace(current_letter, " %s " % current_letter))

        # calculate the current accuracy
        if len(endresult) > 0:
            logger.debug(TRUE_LABELS)
            logger.debug("".join(endresult))
            acc = np.count_nonzero(np.array(endresult) == np.array(list(TRUE_LABELS.lower()[:len(endresult)]))) / len(endresult)
            print("Current accuracy:", acc * 100)
        if len(endresult) == len(TRUE_LABELS):
            break
        #logger.debug("Result: %s" % result)
        print('time',1000 * (time.time() - t0))

    acc = np.count_nonzero(np.array(endresult) == np.array(list(TRUE_LABELS.lower()[:len(endresult)]))) / len(endresult)
    print ("Accuracy:", acc * 100)

    amp.stop()


# 1. Training (off-line)

In [103]:
clf = train(TRAIN_DATA)

# 2. Online experiment

In [211]:
bbuffer = BlockBuffer(12)
cnt = load_bci_data(TEST_DATA)

In [213]:
amp = RReeplayAmp()
amp.configure(data=cnt.data, marker=cnt.markers, channels=cnt.axes[-1], fs=cnt.fs,
              realtime=True, blocksize_samples=12)

In [ ]:
online_experiment(amp, clf)

DEBUG:__main__:Markers processed: 
DEBUG:__main__:0


time 4.960060119628906


DEBUG:__main__:Markers processed: 
DEBUG:__main__:1
DEBUG:__main__:Markers processed: 
DEBUG:__main__:2


time 4.49061393737793
time 4.958868026733398


DEBUG:__main__:Markers processed: 
DEBUG:__main__:3
DEBUG:__main__:Markers processed: 
DEBUG:__main__:4


...

DEBUG:__main__:Markers processed: 
DEBUG:__main__:126
DEBUG:__main__:WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU
DEBUG:__main__:wqxplzcimrkow7yfzdez1dpi9nnvgrpdjcuvrmeuooojd2ufypooj
DEBUG:__main__:Markers processed: 
DEBUG:__main__:127
DEBUG:__main__:WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU
DEBUG:__main__:wqxplzcimrkow7yfzdez1dpi9nnvgrpdjcuvrmeuooojd2ufypooj


Current accuracy: 92.45283018867924
time 7.435083389282227
Current accuracy: 92.45283018867924
time 6.446361541748047


DEBUG:__main__:Markers processed: 
DEBUG:__main__:128
DEBUG:__main__:WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU
DEBUG:__main__:wqxplzcimrkow7yfzdez1dpi9nnvgrpdjcuvrmeuooojd2ufypooj
DEBUG:__main__:Markers processed: 
DEBUG:__main__:129
DEBUG:__main__:WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU
DEBUG:__main__:wqxplzcimrkow7yfzdez1dpi9nnvgrpdjcuvrmeuooojd2ufypooj


Current accuracy: 92.45283018867924
time 7.934093475341797
Current accuracy: 92.45283018867924
time 6.942272186279297


DEBUG:__main__:Markers processed: 
DEBUG:__main__:130
DEBUG:__main__:WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU
DEBUG:__main__:wqxplzcimrkow7yfzdez1dpi9nnvgrpdjcuvrmeuooojd2ufypooj


In [210]:


from __future__ import division

import time

import numpy as np

from libmushu.amplifier import Amplifier


class RReeplayAmp(Amplifier):

    def configure(self, data, marker, channels, fs, realtime=True, blocksize_ms=None, blocksize_samples=None):
        """

        Parameters
        ----------
        data
        marker
        channels
        fs
        realtime
        blocksize_ms : float
            blocksize in milliseconds
        blocksize_samples : int
            blocksize in samples

        Raises
        ------
        TypeError :
            if ``blocksize_ms`` and ``blocksize_s`` are given.

        """
        if [blocksize_ms, blocksize_samples].count(None) != 1:
            raise TypeError("blocksize_ms and blocksize_samples are mutually exclusive.")

        self.data = data
        # slow python
        self.marker = marker
        # fast numpy
        self.marker_ts = np.array([ts for ts, s in marker])
        self.marker_s = np.array([s for ts, s in marker])
        self.channels = channels
        self.fs = fs
        self.realtime = realtime

        if blocksize_ms:
            samples = fs * (blocksize_ms / 1000)
            if not samples.is_integer():
                raise ValueError("Resulting blocksize is not integer, please fix the blocksize_ms.")
            self.samples = samples
        if blocksize_samples:
            self.samples = blocksize_samples

    def start(self):
        self.last_sample_time = time.time()
        self.pos = 0

    def stop(self):
        pass

    def get_data(self):
        """

        Returns
        -------
        chunk, markers: Markers is time in ms since relative to the
        first sample of that block.

        """
        if self.realtime:
            elapsed = time.time() - self.last_sample_time
            blocks = (self.fs * elapsed) // self.samples
            samples = int(blocks * self.samples)
            
        else:
            samples = self.samples
      
        elapsed = samples / self.fs
        self.last_sample_time += elapsed
        # data
        chunk = self.data[self.pos:int(self.pos+samples)]
        #self.data = self.data[samples:]
        # markers

        # slow python version
        #markers = [x for x in self.marker if x[0] < elapsed * 1000]
        #self.marker = [x for x in self.marker if x[0] >= elapsed * 1000]
        #self.marker = [[x[0] - elapsed * 1000, x[1]] for x in self.marker]

        # fast numpy version
        mask = self.marker_ts < (elapsed * 1000) 
        markers = list(zip(self.marker_ts[mask], self.marker_s[mask]))
        self.marker_ts = self.marker_ts[~mask]
        self.marker_s = self.marker_s[~mask]
        self.marker_ts -= elapsed * 1000

        self.pos += samples
        return chunk, markers

    def get_channels(self):
        return self.channels

    def get_sampling_frequency(self):
        return self.fs

    @staticmethod
    def is_available():
        return True
